In [ ]:
import collections
import numpy as np
import pandas as pd
import re

from argparse import Namespace

In [ ]:
# 만약 코랩에서 실행하는 경우 아래 코드를 실행하여 전처리된 라이트 버전의 데이터를 다운로드하세요.
%cd /content/drive/MyDrive/Github_NLP/NLP_with_PyTorch/Chapter4_FFNN_for_NLP/data/
!wget https://git.io/JtaFp -O download.py
!wget https://git.io/Jtabe -O get-all-data.sh
!chmod 755 /content/drive/MyDrive/Github_NLP/NLP_with_PyTorch/Chapter4_FFNN_for_NLP/data/get-all-data.sh
%cd /content/drive/MyDrive/Github_NLP/NLP_with_PyTorch/Chapter4_FFNN_for_NLP/data/
!./get-all-data.sh
%cd ..

/content/drive/MyDrive/Github_NLP/NLP_with_PyTorch/Chapter4_FFNN_for_NLP/data
--2024-03-13 14:24:20--  https://git.io/JtaFp
Resolving git.io (git.io)... 140.82.114.22
Connecting to git.io (git.io)|140.82.114.22|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/rickiepark/nlp-with-pytorch/main/chapter_4/4_2_mlp_surnames/data/download.py [following]
--2024-03-13 14:24:21--  https://raw.githubusercontent.com/rickiepark/nlp-with-pytorch/main/chapter_4/4_2_mlp_surnames/data/download.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1572 (1.5K) [text/plain]
Saving to: ‘download.py’

download.py         100%[===================>]   1.54K  --.-KB/s    in 0s      

2024-03-13 14:24:21 (11.1 M

In [ ]:
base_path = "/content/drive/MyDrive/Github_NLP/NLP_with_PyTorch/Chapter4_FFNN_for_NLP/"

args = Namespace(
    raw_dataset_csv=base_path + "data/surnames/surnames.csv",
    train_proportion=0.7,
    val_proportion=0.15,
    test_proportion=0.15,
    output_munged_csv=base_path + "data/surnames/surnames_with_splits.csv",
    seed=1337
)

In [ ]:
surnames = pd.read_csv(args.raw_dataset_csv, header=0)
print(surnames.head())

    surname nationality
0  Woodford     English
1      Coté      French
2      Kore     English
3     Koury      Arabic
4    Lebzak     Russian


In [25]:
set(surnames.nationality) #총 18개의 클래스

{'Arabic',
 'Chinese',
 'Czech',
 'Dutch',
 'English',
 'French',
 'German',
 'Greek',
 'Irish',
 'Italian',
 'Japanese',
 'Korean',
 'Polish',
 'Portuguese',
 'Russian',
 'Scottish',
 'Spanish',
 'Vietnamese'}

In [26]:
by_nationality = collections.defaultdict(list)
for _, row in surnames.iterrows():
  by_nationality[row.nationality].append(row.to_dict())

for i, dicts in enumerate(by_nationality.items()):
  print(dicts[0], end=": ")
  l = dicts[1]
  print(l[:5]) #예시로 5개만 출력

English: [{'surname': 'Woodford', 'nationality': 'English'}, {'surname': 'Kore', 'nationality': 'English'}, {'surname': 'Essop', 'nationality': 'English'}, {'surname': 'Jefferson', 'nationality': 'English'}, {'surname': 'Dorrington', 'nationality': 'English'}]
French: [{'surname': 'Coté', 'nationality': 'French'}, {'surname': 'Tremblay', 'nationality': 'French'}, {'surname': 'Salvage', 'nationality': 'French'}, {'surname': 'Fontaine', 'nationality': 'French'}, {'surname': 'Cousineau', 'nationality': 'French'}]
Arabic: [{'surname': 'Koury', 'nationality': 'Arabic'}, {'surname': 'Rahal', 'nationality': 'Arabic'}, {'surname': 'Mifsud', 'nationality': 'Arabic'}, {'surname': 'Srour', 'nationality': 'Arabic'}, {'surname': 'Boulos', 'nationality': 'Arabic'}]
Russian: [{'surname': 'Lebzak', 'nationality': 'Russian'}, {'surname': 'Chekhoev', 'nationality': 'Russian'}, {'surname': 'Rosenbloom', 'nationality': 'Russian'}, {'surname': 'Valendik', 'nationality': 'Russian'}, {'surname': 'Lysihin', '

In [27]:
final_list = []
np.random.seed(args.seed)
for _, item_list in sorted(by_nationality.items()):
  np.random.shuffle(item_list)
  n = len(item_list)
  n_train = int(args.train_proportion*n)
  n_val = int(args.val_proportion*n)
  n_test = int(args.test_proportion*n)

  for item in item_list[:n_train]:
    item["split"] = "train"
  for item in item_list[n_train:n_train + n_val]:
    item["split"] = "val"
  for item in item_list[n_train+n_val:]:
    item["split"] = "test"
  final_list.extend(item_list)

In [28]:
final_surnames = pd.DataFrame(final_list)
print(final_surnames.split.value_counts())
final_surnames.head()


train    7680
test     1660
val      1640
Name: split, dtype: int64


,surname,nationality,split
0,Totah,Arabic,train
1,Abboud,Arabic,train
2,Fakhoury,Arabic,train
3,Srour,Arabic,train
4,Sayegh,Arabic,train


In [30]:
print(final_surnames.nationality.value_counts())

English       2972
Russian       2373
Arabic        1603
Japanese       775
Italian        600
German         576
Czech          414
Spanish        258
Dutch          236
French         229
Chinese        220
Irish          183
Greek          156
Polish         120
Korean          77
Scottish        75
Vietnamese      58
Portuguese      55
Name: nationality, dtype: int64


In [ ]:
final_surnames.tail()

,surname,nationality,split
10975,Dinh,Vietnamese,test
10976,Phung,Vietnamese,test
10977,Quang,Vietnamese,test
10978,Vu,Vietnamese,test
10979,Ha,Vietnamese,test


In [ ]:
final_surnames.to_csv(args.output_munged_csv, index=False)